<a href="https://colab.research.google.com/github/Abdullah12389/Multimart/blob/main/vernelable_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests as re
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
data=pd.DataFrame()
for chunk in tqdm(pd.read_csv("/content/myntra202305041052.csv",chunksize=1000,on_bad_lines='skip',nrows=20000)):
  data=pd.concat([data,chunk])

20it [00:00, 41.48it/s]


In [ ]:
data=data.sample(2000)
data=data.loc[:,'name':"img"]

In [ ]:
data=data.reset_index(drop=True)

In [ ]:
words=np.array([])
for word in data["name"]:
  word=word.split()
  words=np.append(words,word[-1])

In [ ]:
dat=pd.concat([data.drop(columns=["name"]),pd.DataFrame(words,columns=["label"])],axis=1)

In [ ]:
for i,word in enumerate(dat["label"]):
  if word.isdigit():
    dat.drop(index=i,inplace=True)

In [ ]:
condition=dat["label"].value_counts()>30
dat["label"].value_counts()[condition]

,count
label,
T-shirt,139
Set,105
Jeans,63
Watch,61
Backpack,59
Dress,56
ml,48
suit,40
Saree,39


In [ ]:
for i in data['name']:
  if i.split()[-1]=="ml":
    print(i)

Strobe Dewy Skin Tint 30 ml
Cream Skin Refiner 150 ml
Stretch Mark Oil 200 ml
Bronze Body Perfume 120 ml
Men Oud Extreme EDP - 100 ml
Tranquil Body Wash 470 ml
Green Tea Daylight Sustainable Sunscreen Gel 50 ml
SPF 15 Body Lotion 200 ml
Baby Daily Lotion - 400 ml
Makeup Setting Spray - 60 ml
Aloe Vera Gel 300 ml
Unisex Sustainable Face Lotion 50 ml
Men Dream Big EDT 100 ml
Revitalift Night Cream 50 ml
Handy 850 Water Bottle 780 ml
Sustain Pure Rose Water 200 ml
Men Intimately 150 ml
Retinol Night Cream - 60 ml
Men Deep Impact Roll-On 50 ml
Men Silver Scent EDT 100 ml
Nayaab Eau De Parfum - 100 ml
Men Xpressio EDP 100 ml
Women Eau de Toilette 30 ml
Eladi Hydrating Cream 30 ml
Men Raw Eau de Parfum 20 ml
Men Classic Perfume - 100 ml
Women Pure Musc EDP 50 ml
Water Bottle 750 ml
Women Eau de Parfum 100 ml
Gentle Skin Cleanser - 125 ml
Unisex Sustainable Face Lotion 50 ml
Micellar Water 400 ml
Shower Wash 200 ml
Women Sheer EDP 20 ml
Men Classic Blue EDT 90 ml
Good Girl Eau de Parfum 50 ml

#I am first downloading images for vernelable product detection

In [ ]:
import os
j=0
folder="images"
for i in data['img'][0:500]:
  response=re.get(i)
  if response.status_code==200:
    file=f"image{j}.jpg"
    with open(f"{folder}/{file}","wb") as f:
      f.write(response.content)
    j+=1
    print("Success")
  else:
     print("fail")

Success
fail
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Suc

In [ ]:
file=os.path.join(folder,"img1.jpg")

In [ ]:
path=data['img'][1]

In [ ]:
response=re.get(path)
with open("images/img1.jpg","wb") as f:
  f.write(response.content)

#Training

In [ ]:
!pip install tensorflow==2.13 onnxruntime tf2onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.2/524.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 26.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  A

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout,GlobalAveragePooling2D

In [ ]:
train_ds=image_dataset_from_directory(
    directory="/content/drive/MyDrive/data/train",
    labels="inferred",
    label_mode="int",
    batch_size=16,
    image_size=(256,256),
    shuffle=True
)

Found 978 files belonging to 2 classes.


In [ ]:
val_ds=image_dataset_from_directory(
    directory="/content/drive/MyDrive/data/test",
    labels="inferred",
    label_mode="int",
    batch_size=16,
    image_size=(256,256),
    shuffle=True
)

Found 118 files belonging to 2 classes.


In [ ]:
def preprocess(img):
  img=img/255.0
  return img
train_ds=train_ds.map(lambda x,y:(preprocess(x),y))
val_ds=val_ds.map(lambda x,y:(preprocess(x),y))

In [ ]:
from tensorflow.keras.applications import EfficientNetV2B3
model=EfficientNetV2B3(include_top=False,
                       weights="imagenet",
                       input_shape=(256,256,3),
                       classes=2,
                       include_preprocessing=False
)

52606240/52606240 [==============================] - 0s 0us/step


In [ ]:
for i in model.layers:
  i.trainable=False

In [ ]:
x=GlobalAveragePooling2D()(model.output)
x=Dense(512,activation="relu")(x)
x=Dropout(0.2)(x)
x=Dense(256,activation="relu")(x)
x=Dropout(0.2)(x)
x=Dense(1,activation="sigmoid")(x)
new_model=tf.keras.models.Model(inputs=model.input,outputs=x)

In [ ]:
new_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
history=new_model.fit(train_ds,validation_data=val_ds,epochs=1)

62/62 [==============================] - 145s 2s/step - loss: 0.0072 - accuracy: 0.9969 - val_loss: 0.1070 - val_accuracy: 0.9831


In [ ]:
new_model.save("vernelable_detect.h5")

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tf2onnx
import onnxruntime as rt
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
signature=[tf.TensorSpec([None,256,256,3],tf.float32,name="input")]

In [ ]:
model=load_model("/content/vernelable_detect.h5")

In [ ]:
onnx_model,_=tf2onnx.convert.from_keras(model,signature,opset=13,output_path="vernelable_detect.onnx")

In [ ]:
output_names=[n.name for n in onnx_model.graph.output]
output_names

['dense_2']

In [ ]:
import numpy as np
from PIL import Image
img=Image.open("/content/drive/MyDrive/data/test/harmful/12Bore (1).png")
img=img.resize((256,256))
img=np.array(img).astype(np.float32)/255
img=np.expand_dims(img,axis=0)

In [ ]:
import onnxruntime as ort
m=ort.InferenceSession("vernelable_detect.onnx",providers=["CPUExecutionProvider"])

In [ ]:
m.run(output_names,{"input":img})

[array([[6.175041e-05]], dtype=float32)]

array([[[0.8627451 , 0.87058824, 0.8666667 ],
        [0.8627451 , 0.87058824, 0.8666667 ],
        [0.85882354, 0.8666667 , 0.8627451 ],
        ...,
        [0.3137255 , 0.3137255 , 0.29411766],
        [0.30980393, 0.3137255 , 0.30588236],
        [0.29803923, 0.30588236, 0.29411766]],

       [[0.8745098 , 0.88235295, 0.8784314 ],
        [0.8745098 , 0.88235295, 0.8784314 ],
        [0.8745098 , 0.88235295, 0.8784314 ],
        ...,
        [0.30588236, 0.30588236, 0.28627452],
        [0.28235295, 0.29803923, 0.28235295],
        [0.2627451 , 0.27058825, 0.25882354]],

       [[0.8862745 , 0.8980392 , 0.89411765],
        [0.8862745 , 0.8980392 , 0.89411765],
        [0.8862745 , 0.89411765, 0.89411765],
        ...,
        [0.2901961 , 0.2901961 , 0.27450982],
        [0.2509804 , 0.27058825, 0.25882354],
        [0.21568628, 0.22745098, 0.21568628]],

       ...,

       [[0.08235294, 0.07843138, 0.05882353],
        [0.08627451, 0.08235294, 0.0627451 ],
        [0.09019608, 0